In [250]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display
import keras
from keras.layers import Dense, Activation
from keras.models import Sequential
import tensorflow as tf

In [256]:
import gym

env = gym.make("FrozenLake8x8-v0")
env.reset()

#plt.imshow(env.render('rgb_array'))
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)
print('observation space:', env.observation_space)
print('action space:', env.action_space)
print("taking action")
new_obs, reward, is_done, _ = env.step(1)

print("new observation code:", new_obs)
print("reward:", reward)
print("is game over?:", is_done)


Observation space: Discrete(64)
Action space: Discrete(4)
observation space: Discrete(64)
action space: Discrete(4)
taking action
new observation code: 0
reward: 0.0
is game over?: False


In [257]:
n_states = env.observation_space.n
n_actions = env.action_space.n
print("Number of States: ", n_states)
print("Number of Actions: ", n_actions)

Number of States:  64
Number of Actions:  4


In [258]:
#Input policy to be evaluated

policy = np.ones((n_states , n_actions))/n_actions
print("Shape of Policy Matrix" , policy.shape)

#Initialize array V(s) = 0

V = np.zeros(n_states)
discount_factor = 1.0
theta = 0.00001

Shape of Policy Matrix (64, 4)


In [259]:
env.P[3][3]

[(0.3333333333333333, 4, 0.0, False),
 (0.3333333333333333, 3, 0.0, False),
 (0.3333333333333333, 2, 0.0, False)]

### Policy Evaluation

In [260]:
#Repeat:
while True:
    delta = 0.0        #delta <- 0
    
    for s in range(n_states):   #For each state:
        v = 0
        for a , policy_prob in enumerate(policy[s]):                            #Expression---
            for prob, next_state, reward, done  in env.P[s][a]:
                v += policy_prob * prob *(reward + discount_factor*V[next_state])     # V(s) = sum(policy(a|s) * sum(prob(s', r|s,a)) * (reward + gamma*V(s')))))
        print(abs(v - V[s]))                                                         #---
        delta = max(delta , abs(v - V[s]))            #delta <- max(delta , |v - V[s]|)
        V[s] = v

    if(delta<theta):                    #terminate on delta < theta (theta is a small positive number)
        break
# Output V ~= vpi

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.25
0.0
0.0
0.0
0.0
0.0
0.0
0.25
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0625
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.07812499999999994
0.0
0.0
0.0
0.0
0.0
0.0625
0.07812499999999994
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.015625
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.03906249999999996
0.0
0.0
0.0
0.0
0.0
0.015625
0.0
0.029296875000000056
0.0
0.0
0.0
0.0
0.015625
0.04296874999999997
0.0302734375
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0

0.00016398311191664905
0.00023339037385890216
7.264945728755376e-06
7.133916178808391e-06
5.8382494939187505e-06
0.0
2.245237558531031e-05
3.450021934883925e-05
8.83896139265948e-05
0.00013529617769658664
2.6857774950703486e-06
0.0
0.0
3.156470960808365e-06
9.993634867906317e-06
1.2304325793119353e-05
0.0
5.56073726785955e-05
1.1228026068903138e-06
0.0
3.2711258175000576e-07
8.708958856396367e-07
0.0
3.8817751245867615e-06
0.0
1.9602771784144046e-05
7.527960888653941e-07
3.510773100734064e-07
2.3842884825068e-07
0.0
1.2606369335643741e-06
2.4064559810688113e-06
9.167732855086541e-07
0.0
3.587770520015559e-05
5.669233635151582e-05
9.633293650461558e-05
0.00015987860091263065
0.00025319723351645805
0.0003541947272655494
0.0004420825820253653
0.000491893539413887
3.2231412048329996e-05
4.6501158232664365e-05
7.115893021983582e-05
0.00010977489859393145
0.000195762282833667
0.00028435915685737917
0.00036849381268744784
0.0004169186210661105
2.3271387862738686e-05
2.841452984192759e-05
2.89

1.3623783658183852e-05
1.9731913933705887e-05
2.8119407147357556e-05
3.274933843599656e-05
1.3628815793980775e-05
1.1263381324379154e-05
7.55683438951086e-06
3.1822640007175655e-06
4.7979462237997955e-06
0.0
1.456782087380648e-05
2.0395889851101967e-05
7.69932618449842e-06
5.606687269137664e-06
3.290880414662104e-06
0.0
2.207193870189597e-06
2.827790746168582e-06
7.38044886611916e-06
1.1168700366312456e-05
3.0848410968375636e-06
0.0
0.0
2.623841470962996e-07
8.904717584906977e-07
1.007579862703778e-06
0.0
4.394716315636593e-06
1.3577082813684318e-06
0.0
5.7302416102024e-08
7.992164079959107e-08
0.0
2.878298449721095e-07
0.0
1.5081505770897508e-06
8.956247972249614e-07
3.506909746353937e-07
1.377934467867978e-07
0.0
4.9307673596898915e-08
1.32546177311621e-07
4.5463462716721637e-08
0.0
3.0762260790595e-05
2.9460180562044082e-05
2.883614786969085e-05
3.0134397739904183e-05
3.528381670720674e-05
4.157784452741252e-05
4.7250973333304996e-05
5.051400369246277e-05
2.5909407891432177e-05
2.39

In [242]:
V

array([1.77291852e-03, 2.05231760e-03, 2.71200668e-03, 4.02682148e-03,
       6.45060006e-03, 9.69730771e-03, 1.33323211e-02, 1.58487758e-02,
       1.52729353e-03, 1.69427642e-03, 2.07670686e-03, 2.93669579e-03,
       5.64863740e-03, 9.33260477e-03, 1.44768170e-02, 1.83922528e-02,
       1.13470573e-03, 1.13113238e-03, 9.71534634e-04, 0.00000000e+00,
       3.88354185e-03, 7.51853274e-03, 1.68629320e-02, 2.48647709e-02,
       7.60085659e-04, 7.30250376e-04, 6.80298040e-04, 7.62713639e-04,
       2.37149567e-03, 0.00000000e+00, 2.05998575e-02, 3.93483800e-02,
       4.24537163e-04, 3.53058351e-04, 2.58339098e-04, 0.00000000e+00,
       4.84014067e-03, 1.15878559e-02, 2.61929417e-02, 7.25859868e-02,
       1.65174830e-04, 0.00000000e+00, 0.00000000e+00, 1.44772835e-03,
       5.40187309e-03, 1.53198027e-02, 0.00000000e+00, 1.52219331e-01,
       7.23056825e-05, 0.00000000e+00, 1.09152955e-04, 3.89220327e-04,
       0.00000000e+00, 4.42895328e-02, 0.00000000e+00, 3.84073024e-01,
      